# El Hola Mundo del Aprendizaje Profundo con Redes Neuronales
Basado en https://colab.research.google.com/github/lmoroney/mlday-tokyo/blob/master/Lab1-Hello-ML-World.ipynb#scrollTo=X9uIpOS2zx7k
Originalmente en: https://github.com/lmoroney/mlday-tokyo/blob/master/Lab1-Hello-ML-World.ipynb

Como con cualquier primera aplicación, deberías comenzar con algo súper simple que muestre el andamiaje general de cómo funciona tu código.
En el caso de crear redes neuronales, el ejemplo que me gusta usar es uno donde aprende la relación entre dos números. Entonces, por ejemplo, si estuvieras escribiendo código para una función como esta, ya conoces las "reglas":

'''
float my_function(float x){
    float y = (3 * x) + 1;
    return y;
}
'''
Entonces, ¿cómo entrenarías una red neuronal para realizar la tarea equivalente? ¡Usando datos! Al alimentarla con un conjunto de X y un conjunto de Y, debería ser capaz de descubrir la relación entre ellos.
Este es obviamente un paradigma muy diferente al que podrías estar acostumbrado, así que vamos a analizarlo pieza por pieza.

# Importaciones
Comencemos con nuestras importaciones.

'jax.numpy' se importa como jnp, que proporciona una interfaz similar a NumPy para los arrays de JAX.

'grad', 'jit' y 'vmap' se importan del módulo jax: 'grad' se utiliza para la diferenciación automática, 'jit' se utiliza para la compilación just-in-time y 'vmap' se utiliza para vectorizar funciones.

'optimizers' se importa de 'jax.example_libraries', que proporciona una colección de algoritmos de optimización.

In [22]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.example_libraries import optimizers

# Definir y Compilar la Red Neuronal
A continuación, crearemos la red neuronal más simple posible.
Toma dos parámetros: 'params' (los parámetros del modelo) y 'x' (los datos de entrada).
El modelo simplemente multiplica params por x y devuelve el resultado.

In [23]:
# Define the neural network
def model(params, x):
    return params * x

Ahora compilamos nuestra Red Neuronal. Al hacerlo, tenemos que especificar 2 funciones, una pérdida y un optimizador.

Si has visto mucha matemática para machine learning, aquí es donde generalmente se usa, pero en este caso está muy bien encapsulada en funciones para ti. Pero qué sucede aquí, expliquemos...

Sabemos que en nuestra función, la relación entre los números es y=3x+1.

Cuando la computadora está tratando de "aprender" eso, hace una suposición... tal vez y=10x+10. La función de PÉRDIDA mide las respuestas supuestas contra las respuestas correctas conocidas y mide qué tan bien o mal lo hizo.

Luego usa la función de OPTIMIZADOR para hacer otra suposición. Según cómo fue la función de pérdida, intentará minimizar la pérdida. En ese punto, tal vez llegue a algo como y=5x+5, que, aunque todavía es bastante malo, está más cerca del resultado correcto (es decir, la pérdida es menor).

Repetirá esto por el número de ÉPOCAS que verás en breve. Pero primero, así es como le decimos que use 'ERROR CUADRÁTICO MEDIO' para la pérdida y 'DESCENSO DE GRADIENTE ESTOCÁSTICO' para el optimizador. Aún no necesitas entender la matemática de estos, ¡pero puedes ver que funcionan! :)

Con el tiempo aprenderás las diferentes funciones de pérdida y optimizador apropiadas para diferentes escenarios.


La función 'loss'  define la función de pérdida utilizada para medir la diferencia entre los valores predichos y los valores reales.

Toma tres parámetros: 'params' (los parámetros del modelo), 'x' (los datos de entrada) e 'y' (los valores de salida reales).

Llama a la función 'model' para obtener los valores predichos y_pred.

Calcula el error cuadrático medio entre 'y_pred' e 'y' utilizando 'jnp.mean((y_pred - y) ** 2)'.

La función 'step' define el paso de optimización.

Está decorada con '@jit', lo que indica que la función debe ser compilada just-in-time para una ejecución más rápida.

Toma cuatro parámetros: 'i' (la iteración actual), 'opt_state' (el estado del optimizador), 'x' (los datos de entrada) e 'y' (los valores de salida reales).

Recupera los parámetros actuales del modelo del estado del optimizador utilizando 'get_params(opt_state)'.

Calcula los gradientes de la función de pérdida con respecto a los parámetros utilizando 'grad(loss)(params, x, y)'.

Actualiza el estado del optimizador utilizando 'opt_update(i, g, opt_state)', donde 'i' es la iteración actual, 'g' son los gradientes y 'opt_state' es el estado actual del optimizador.

In [24]:
# Define the loss function
def loss(params, x, y):
    y_pred = model(params, x)
    return jnp.mean((y_pred - y) ** 2)

# Define the optimization step
@jit
def step(i, opt_state, x, y):
    params = get_params(opt_state)
    g = grad(loss)(params, x, y)
    return opt_update(i, g, opt_state)

# Proporcionar los Datos
A continuación, introduciremos algunos datos. En este caso estamos tomando 6 x y 6 y. Puedes ver que la relación entre estos es que y=2x-1, entonces donde x = -1, y=-3, etc.

Los datos se crean como arrays de JAX utilizando jnp.array().


In [25]:
# Provide the data
xs = jnp.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0])
ys = jnp.array([-2.0, 1.0, 4.0, 7.0, 10.0, 13.0])

# Entrenamiento de la Red Neuronal
El proceso de entrenar la red neuronal, donde "aprende" la relación entre las X y las Y está en la llamada model.fit. Aquí es donde pasará por el bucle del que hablamos anteriormente, haciendo una suposición, midiendo qué tan buena o mala es (también conocida como pérdida), usando el optimizador para hacer otra suposición, etc. Lo hará por el número de épocas que especifiques.

'params = jnp.array([0.0])'

Esta línea inicializa los parámetros del modelo como un array de JAX con un único valor de 0.0.

'''
opt_init, opt_update, get_params = optimizers.sgd(step_size=0.01)
opt_state = opt_init(params)
'''

Estas líneas crean el optimizador utilizando el algoritmo de descenso de gradiente estocástico (SGD) de 'optimizers.sgd'.

El parámetro 'step_size' especifica la tasa de aprendizaje (learning rate) del optimizador.

'opt_init' es la función utilizada para inicializar el estado del optimizador, 'opt_update' es la función utilizada para actualizar el estado del optimizador y 'get_params' es la función utilizada para recuperar los parámetros actuales del estado del optimizador.

El estado del optimizador se inicializa utilizando 'opt_init(params)', donde 'params' son los parámetros iniciales del modelo.


In [26]:
# Initialize the parameters
params = jnp.array([0.0])

# Create the optimizer
opt_init, opt_update, get_params = optimizers.sgd(step_size=0.01)
opt_state = opt_init(params)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    opt_state = step(epoch, opt_state, xs, ys)
    if (epoch + 1) % 50 == 0:
        params = get_params(opt_state)
        loss_value = loss(params, xs, ys)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss_value:.4f}")

Epoch [50/50], Loss: 0.5655


Bien, ahora tienes un modelo que ha sido entrenado para aprender la relación entre X e Y. Puedes usar el método model.predict para hacer que determine la Y para una X previamente desconocida. Entonces, por ejemplo, si X = 10, ¿cuál crees que será Y? Adivina antes de ejecutar este código:

In [27]:
new_x = jnp.array([10.0])
print(new_x)

[10.]


In [28]:
# Make a prediction
predicted_y = model(params, new_x)
print(f"Predicted Y for X = 10: {predicted_y.item():.4f}")

Predicted Y for X = 10: 32.7624


Podrías haber pensado 31, ¿verdad? Pero terminó siendo un poco más. ¿Por qué crees que es eso?
Recuerda que las redes neuronales trabajan con probabilidades, así que dados los datos que alimentamos a la RN con, calculó que hay una probabilidad muy alta de que la relación entre X e Y sea Y=3X+1, pero con solo 6 puntos de datos no podemos estar seguros. Como resultado, el resultado para 10 es muy cercano a 31, pero no necesariamente 31.
A medida que trabajes con redes neuronales, verás este patrón recurrente. Casi siempre tratarás con probabilidades, no certezas, y harás un poco de codificación para descubrir cuál es el resultado basado en las probabilidades, particularmente cuando se trata de clasificación.

Después del entrenamiento, se crea un nuevo valor de entrada (new_x) como un array de JAX con un valor de 10.0.

Se utiliza el modelo entrenado para hacer una predicción para new_x llamando a model(params, new_x), donde params son los parámetros finales entrenados.

El valor predicho (predicted_y) se imprime utilizando una cadena f-string con el método .item() para extraer el valor escalar del array de JAX y el especificador de formato .4f para mostrar el valor con cuatro decimales.